In [11]:
import pandas as pd
import json
import nltk
import numpy as np
from nltk.corpus import stopwords
import string

In [2]:
data = pd.read_csv('data/restaurant_reviews.csv',dtype=object)
data.head()
data.columns
print(len(data['text']))

5055992


In [36]:
reviews = data.sample(n=150000,random_state=42)
review_text = reviews['text']
stars = reviews['stars']
reviews.head()


,business_id,stars,useful,funny,cool,text,date
988416,_HW7sgUShin0aGouFMiUdA,5.0,2.0,0.0,0.0,I went here with a group of friends for the $5...,2011-02-13 22:00:30
4635308,PQER4ba8Q0zqB1G2QiXamQ,3.0,0.0,0.0,0.0,I came to Black Bear Diner while on a little 5...,2015-09-01 15:48:14
1418214,3U__XOTBFptjhBeHrPGqrg,4.0,5.0,2.0,2.0,Yum...this place is delicious! I stopped in f...,2015-08-02 19:11:39
4908073,sPwgafejYVdf8hd4-oDYVg,1.0,1.0,1.0,0.0,"Sure, it's Super Bowl Sunday, but taking the p...",2017-02-05 21:27:06
4984700,umXvdus9LbC6oxtLdXelFQ,5.0,2.0,1.0,3.0,Seinaru kemuri!!! That's it... all I have to s...,2019-02-04 05:26:15


In [37]:
def clean_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    cleaned_text = [word for word in nopunc.split() 
                    if word.lower() not in stopwords.words('english')]
    return cleaned_text

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer=clean_text)
tfidf_vectorizer.fit(review_text)
X = tfidf_vectorizer.transform(review_text)

In [69]:

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, stars, train_size = 0.7
)

In [45]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.48930666666666667
Accuracy for C=0.05: 0.59496


C:\Users\Shehryar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy for C=0.25: 0.6377333333333334


C:\Users\Shehryar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy for C=0.5: 0.6415733333333333
Accuracy for C=1: 0.6424533333333333


C:\Users\Shehryar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [70]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print("Accuracy for C=%s: %s" % (c, accuracy_score(y_test, svm.predict(X_test))))

Accuracy for C=0.01: 0.5965555555555555
Accuracy for C=0.05: 0.6285333333333334
Accuracy for C=0.25: 0.6309333333333333
Accuracy for C=0.5: 0.6242888888888889
Accuracy for C=1: 0.6151777777777778


In [72]:
final_model = svm = LinearSVC(C=.25)
svm.fit(X_train, y_train)
print("Accuracy for C=%s: %s" % (1, accuracy_score(y_test, svm.predict(X_test))))
import pickle

#
# Create your model here (same as above)
#

# Save to file in the current working directory
pkl_filename = "shehryar_upadated.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(final_model, file)

Accuracy for C=1: 0.6309333333333333


In [73]:
X2 = pd.DataFrame(X)
print(type(X2))
X2['useful'] = reviews['useful'].to_numpy()
X2['cool'] = reviews['cool'].to_numpy()
X2['funny'] = reviews['funny'].to_numpy()
X2.head()
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, stars, train_size = 0.6
)

<class 'pandas.core.frame.DataFrame'>


In [74]:
X2.head()

,0,useful,cool,funny
0,"(0, 162284)\t0.06254128788923516\n (0, 1607...",2.0,0.0,0.0
1,"(0, 162294)\t0.1153541835131528\n (0, 16228...",0.0,0.0,0.0
2,"(0, 163038)\t0.2116494294634623\n (0, 15301...",5.0,2.0,2.0
3,"(0, 162859)\t0.15017726142107757\n (0, 1622...",1.0,0.0,1.0
4,"(0, 163559)\t0.05150316091957389\n (0, 1634...",2.0,3.0,1.0


In [75]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    svm = LinearSVC(C=c)
    svm.fit(X2_train, y2_train)
    print("Accuracy for C=%s: %s" % (c, accuracy_score(y2_test, svm.predict(X2_test))))

ValueError: setting an array element with a sequence.

In [7]:
reviews = data.sample(frac=.2,random_state=42)
review_text = reviews['text']
stars = reviews['stars']
reviews.head()

def clean_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    cleaned_text = [word for word in nopunc.split() 
                    if word.lower() not in stopwords.words('english')]
    return cleaned_text

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1, 2),analyzer=clean_text)
cv.fit(review_text)


CountVectorizer(analyzer=<function clean_text at 0x0000020410C72948>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [8]:
X2 = cv.transform(review_text)

In [9]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, stars, train_size = 0.7
)

In [10]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    svm = LinearSVC(C=c)
    svm.fit(X2_train, y2_train)
    print("Accuracy for C=%s: %s" % (c, accuracy_score(y2_test, svm.predict(X2_test))))

Accuracy for C=0.01: 0.6405458860759494
Accuracy for C=0.05: 0.6359012394514768
Accuracy for C=0.25: 0.6249044040084388
Accuracy for C=0.5: 0.6181335706751054
Accuracy for C=1: 0.6107100474683544


In [14]:
data['positive'] = data['stars'].apply(
    lambda x: 1 if float(x) >= 3 else 0)
reviews = data.sample(n=200000,random_state=42)
review_text = reviews['text']
stars = reviews['positive']
reviews.head()

def clean_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    cleaned_text = [word for word in nopunc.split() 
                    if word.lower() not in stopwords.words('english')]
    return cleaned_text

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1, 2),analyzer=clean_text)
cv.fit(review_text)
X3 = cv.transform(review_text)
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, stars, train_size = 0.7
)
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    svm = LinearSVC(C=c)
    svm.fit(X3_train, y3_train)
    print("Accuracy for C=%s: %s" % (c, accuracy_score(y3_test, svm.predict(X3_test))))

Accuracy for C=0.01: 0.9195666666666666
Accuracy for C=0.05: 0.9172333333333333
Accuracy for C=0.25: 0.9098833333333334
Accuracy for C=0.5: 0.9061666666666667
Accuracy for C=1: 0.9007666666666667
